In [ ]:
# sat_ppm=-3.45
# sat_freq=sat_ppm*1e-6*B0*sys.gamma

In [1]:
import os
cwd = os.getcwd()
import shutil
from glob import glob
import SimpleITK as sitk
from pydicom import dcmread
from tqdm import trange
import matplotlib.pyplot as plt
import numpy as np

def read_dicom_folder(phi_path):
    delta_phi_dcms = glob("*.IMA", root_dir=phi_path)
    delta_phi_dcms.sort()
    imgs = []
    for dcm in delta_phi_dcms:
        imgs.append(dcmread('{}/{}'.format(phi_path, dcm)).pixel_array)
    return np.array(imgs)

def check_path(p: str) -> str:
    if  not os.path.exists(p):
        os.makedirs(p)
    return p

dcm2niix_path   = 'D:/TOOLS/dcm2niix/dcm2niix.exe'
dicom_path    = '20230921_S17_CSTOF'

In [4]:
f = 297.193848 # MHz
te1 = 0.00306
te2 = 0.00408
delta_t = te2 - te1
e1 = sitk.ReadImage('20230921_S17_CSTOF_rearrange/6_gre_field_mapping_tra_brain_e1.nii.gz')
e2 = sitk.ReadImage('20230921_S17_CSTOF_rearrange/6_gre_field_mapping_tra_brain_e2.nii.gz')

e1_array = sitk.GetArrayFromImage(e1)
e2_array = sitk.GetArrayFromImage(e2)
phi_array = read_dicom_folder(phi_path='20230921_S17_CSTOF_rearrange/7_gre_field_mapping_tra_brain')
phi_array = phi_array /4095*2*np.pi-np.pi # [-pi, pi]

In [5]:
print(phi_array.min(), phi_array.max())

-3.141592653589793 3.141592653589793


In [11]:
B0_array = phi_array / (delta_t * 2 * np.pi)
B0_image = sitk.GetImageFromArray(np.flip(B0_array, axis=1))
B0_image.CopyInformation(e1)
sitk.WriteImage(sitk.Cast(B0_image, sitk.sitkInt16),"B0map_hertz.nii.gz",useCompression=True)
sitk.Cast(B0_image, sitk.sitkFloat32)

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x0000023B8D24BDE0> >

In [39]:
# e1_phi_array = read_phase_dicom_folder(phi_path='20230921_S17_CSTOF_rearrange/13_t2_star_tra_p3_dTE1.05')
phi_path='20230921_S17_CSTOF_rearrange/13_t2_star_tra_p3_dTE1.05'
dcms = glob("*IMA", root_dir=phi_path)
dcms.sort()
phi_dcms_dict = {}
for dcm in dcms:
    ds = dcmread('{}/{}'.format(phi_path, dcm))
    if ds.EchoNumbers not in phi_dcms_dict.keys():
        phi_dcms_dict[ds.EchoNumbers] = []
    phi_dcms_dict[ds.EchoNumbers].append(dcm)

e1_phi_array = []
for dcm in phi_dcms_dict[1]:
    e1_phi_array.append(dcmread('{}/{}'.format(phi_path, dcm)).pixel_array)
e1_phi_array = np.array(e1_phi_array)/4095*2*np.pi-np.pi # [-pi, pi]

e2_phi_array = []
for dcm in phi_dcms_dict[2]:
    e2_phi_array.append(dcmread('{}/{}'.format(phi_path, dcm)).pixel_array)
e2_phi_array = np.array(e2_phi_array)/4095*2*np.pi-np.pi

In [58]:
delta_phi_array = e2_phi_array - e1_phi_array

delta_phi_array = np.where(delta_phi_array<-np.pi, delta_phi_array+2*np.pi, delta_phi_array)
delta_phi_array = np.where(delta_phi_array>np.pi, delta_phi_array-2*np.pi, delta_phi_array)
sitk.WriteImage(sitk.GetImageFromArray(delta_phi_array / (0.00105 * 2 * np.pi)),"B0map_t2_star_tra_p3_dTE1.05.nii.gz")